In [1]:
# Financial Math Cheat Sheet: numpy_financial, QuantLib, FinancePy, pyfinance

"""
This Jupyter Notebook contains examples of financial calculations using:
- numpy_financial: basic TVM functions
- QuantLib: bond pricing, YTM, duration, convexity
- FinancePy: bond analytics
- pyfinance: IRR, NPV, and regression analysis
"""

# ------------------------------------------
# 1. Time Value of Money (TVM) - numpy_financial
# ------------------------------------------
import numpy_financial as npf

# FV: Future Value
fv = npf.fv(rate=0.05, nper=10, pmt=0, pv=-1000)
print("Future Value:", fv)

# PV: Present Value
pv = npf.pv(rate=0.05, nper=10, pmt=0, fv=1628.89)
print("Present Value:", pv)

# PMT: Payment
pmt = npf.pmt(rate=0.05, nper=10, pv=-1000, fv=0)
print("Payment:", pmt)

# NPV: Net Present Value
npv = npf.npv(rate=0.05, values=[-1000, 200, 300, 400, 500])
print("Net Present Value:", npv)

# IRR: Internal Rate of Return
irr = npf.irr(values=[-1000, 300, 400, 500, 600])
print("Internal Rate of Return:", irr)

# ------------------------------------------
# 2. QuantLib - Fixed Income and Yield Calculations
# ------------------------------------------
import QuantLib as ql

# Yield to Maturity (YTM) Calculation
settlement = ql.Date(1, 1, 2024)
maturity = ql.Date(1, 1, 2034)
schedule = ql.Schedule(settlement, maturity, ql.Period(ql.Annual), ql.UnitedStates(),
                       ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)
bond = ql.FixedRateBond(0, ql.USDCurrency(), 1000, schedule, [0.05], ql.ActualActual())
discount_curve = ql.FlatForward(settlement, 0.04, ql.ActualActual())
bond.setPricingEngine(ql.DiscountingBondEngine(ql.YieldTermStructureHandle(discount_curve)))
ytm = bond.bondYield(ql.ActualActual(), ql.Compounded, ql.Annual)
print("QuantLib YTM:", ytm)

# Present Value via discount factor
pv_quantlib = bond.cleanPrice()
print("QuantLib PV:", pv_quantlib)

# Duration and Convexity
bond_duration = ql.BondFunctions.duration(bond, discount_curve, ql.Duration.Modified)
print("QuantLib Duration:", bond_duration)
bond_convexity = ql.BondFunctions.convexity(bond, discount_curve)
print("QuantLib Convexity:", bond_convexity)

# ------------------------------------------
# 3. FinancePy - Fixed Income and Derivatives
# ------------------------------------------
from financepy.products.bond import FixedRateBond
from financepy.utils.date import Date
from financepy.utils.day_count import DayCountType

settlement_date = Date(2024, 1, 1)
maturity_date = Date(2034, 1, 1)
bond_fp = FixedRateBond("US T-Bond", 1000, 0.05, settlement_date, maturity_date,
                         DayCountType.ACT_ACT, 1)

# Price the bond using a flat yield curve
price_fp = bond_fp.price(0.04)
print("FinancePy Price:", price_fp)

# Yield to Maturity (approximate)
ytmt_fp = bond_fp.calc_ytm(price_fp)
print("FinancePy YTM:", ytmt_fp)

# Accrued Interest
accrued_interest_fp = bond_fp.calc_accrued_interest()
print("FinancePy Accrued Interest:", accrued_interest_fp)

# ------------------------------------------
# 4. pyfinance - IRR, NPV, and Performance Analytics
# ------------------------------------------
from pyfinance.ols import TSMRegress
import pandas as pd
import numpy as np

# IRR / NPV example
cash_flows = [-1000, 300, 400, 500, 600]
dates = pd.date_range(start='2024-01-01', periods=len(cash_flows), freq='A')
series = pd.Series(data=cash_flows, index=dates)

irr_py = series.irr()
print("pyfinance IRR:", irr_py)
npv_py = series.npv(rate=0.05)
print("pyfinance NPV:", npv_py)

# Regression example (e.g., CAPM beta)
rets = pd.DataFrame({
    'asset': np.random.normal(0.01, 0.02, 100),
    'market': np.random.normal(0.01, 0.015, 100)
})
model = TSMRegress(rets['asset'], rets[['market']])
beta = model.beta['market']
print("CAPM Beta:", beta)

Future Value: 1628.894626777442
Present Value: -999.9971595600073
Payment: 129.50457496545664
Net Present Value: 219.47131082213673
Internal Rate of Return: 0.2488833566240709


TypeError: UnitedStates.__init__() missing 1 required positional argument: 'm'